In [1]:
import pandas as pd
import sklearn
from sklearn.decomposition import TruncatedSVD
import numpy as np
import warnings
warnings.filterwarnings("ignore")

user_prefer = pd.DataFrame()

user_prefer = pd.read_csv('./lastfm-2k/data/u5.train', sep=',',names=["userID","artistID","weight"],header=None, na_filter=False)
user_cluster = pd.read_csv('./lastfm-2k/data/u.csv', sep=',', na_filter=False)
user_test = pd.read_csv('./lastfm-2k/data/u5.test', sep=',',names=["userID","artistID","weight"],header=None, na_filter=False)

In [2]:
user_name_cluster = user_cluster.pivot_table('weight', index = 'userID',columns = 'artistID').fillna(0)
user_artist_rating = user_prefer.pivot_table('weight', index = 'userID',columns = 'artistID').fillna(0) # train 용
user_test = user_test.pivot_table('weight', index = 'userID',columns = 'artistID').fillna(0)

idx = user_name_cluster.index

#train
# normalization_user_artist_rating = pd.DataFrame(user_artist_rating)
# normalization_user_artist_rating.index = user_artist_rating.index
# normalization_user_artist_rating.columns = user_artist_rating.columns


#cluster용 dataset
SVD = TruncatedSVD(n_components=2) # 2차원으로 생성
# matrix = SVD.fit_transform(user_name_cluster)
# matrix = pd.DataFrame(matrix)
# matrix.index = idx


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from itertools import cycle
import networkx as nx
import matplotlib.pyplot as plt

#social impact용 data
df = pd.read_csv('lastfm-2k/user_friends.csv', sep='\t', na_filter=False)

values = []

for i in df.index:
    if int(df.loc[i][0]) in user_name_cluster.index and int(df.loc[i][1]) in user_name_cluster.index:
        values.append(df.loc[i])

friend_df = pd.DataFrame()
friend_df = pd.DataFrame(values)

G = nx.from_pandas_edgelist(friend_df, 'userID','friendID')

# closeness centrality
# close_centrality = nx.closeness_centrality(G)
# close_centrality = pd.DataFrame.from_dict(close_centrality, orient='index', columns=['closeness'])

# close_centrality_drop = close_centrality[close_centrality.closeness>0.20]

In [3]:
def grs(num, default_matrix, close_centrality_drop, save_rating):
    import numpy as np
    
    matrix = default_matrix
    #h = num/10
    
    normalization_user_artist_rating = save_rating

    #KMeans(Viusalization)
    km = KMeans(n_clusters=10, init='k-means++')
    kcluster = km.fit(matrix)
    kcluster_id = pd.DataFrame(kcluster.labels_)

    d1 = pd.concat([matrix, kcluster_id], axis=1)
    d1.columns = [0, 1, "cluster"]

    kcluster_id.index = matrix.index

    kcluster_id.rename(columns = {0 : 'cluster'}, inplace = True)

        #AVG용
    normalization_user_artist_rating["sum"] = normalization_user_artist_rating.mean(axis=1)
    user_test["sum"] = user_test.mean(axis=1)

    #train, test셋에 cluster 할당
    cluster = normalization_user_artist_rating
    cluster["cluster"] = np.nan

    user_test["cluster"] = np.nan

    for i in kcluster_id.index:
        if i in cluster.index:
            cluster["cluster"][i] = kcluster_id["cluster"][i]
        if i in user_test.index:
            user_test["cluster"][i] = kcluster_id["cluster"][i]

    cluster_user_matrix = pd.DataFrame(cluster)

    #cluster 수 및 cluster별 인원 수 저장하는 list
    length = []
    length.append(user_test["cluster"].value_counts())

    clusters = []

    #user-item 정보 클러스터 별로 저장
    for i in range(len(length[0])):
        clusters.append(cluster_user_matrix[cluster_user_matrix["cluster"]==i])
        
    sum_point_idx = []
    for i in range(len(length[0])):
        sum_point_idx.append(i)
    
    #predict 저장할 DataFrame
    sum_point = pd.DataFrame(index=sum_point_idx, columns = cluster_user_matrix.columns).fillna(0)

    for i in range(10):
        #clusters[i] = clusters[i].replace({'0':np.nan, 0:np.nan})
        sum_point.loc[i] = clusters[i].max(axis=0, skipna=True)
    
    sum_point = sum_point.replace({np.nan:0})
            
    point = sum_point[sum_point.index < len(length[0])]

    scores = point.drop(["cluster"], axis=1)
    cluster_user_matrix.drop(["sum"], axis=1)

    #정답 셋
    tmp = pd.DataFrame(columns = user_test.columns)
    tmp["num"] = np.nan
    #print(tmp)
    #정답 셋 정보 할당
    for i in user_test.index:
        idx = 0
        while idx < len(length[0]):
            if user_test["cluster"][i] == idx:
                tmp.loc[i] = user_test.loc[i]
                tmp["num"][i] = idx
            idx += 1

    number = pd.DataFrame(tmp["num"])

    #평가
    from sklearn.metrics import ndcg_score

    tmp = tmp.drop(["sum","cluster","num"], axis=1)

    #miss matching 제거
    for i in scores.columns:
        if i not in tmp.columns:
            scores = scores.drop([i], axis=1)

    for i in tmp.columns:
        if i not in scores.columns:
            tmp = tmp.drop([i], axis=1)
    
    result = []
    for i in range(len(length[0])):
        result.append(0)

    for i in tmp.index:

        idx = 0

        while idx < len(length[0]):
            if number["num"][i] == idx:
                result[idx] += ndcg_score([tmp.loc[i]], [scores.loc[idx]])
            idx += 1

    for i in range(len(length[0])):
        result[i] = result[i]/length[0][i]


    #print(length[0])
    print("cluster수:",len(length[0]),"\thyper param: 0.84/-0.94뺌","/ NDCG:",sum(result)/len(length[0]))

In [4]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

cluster_num = [1]

for i in cluster_num:
    close_centrality = nx.closeness_centrality(G)
    close_centrality = pd.DataFrame.from_dict(close_centrality, orient='index', columns=['closeness'])
    close_centrality_drop = close_centrality[close_centrality.closeness>0.20]
    matrix = SVD.fit_transform(user_name_cluster)
    matrix = pd.DataFrame(matrix)
    matrix.index = idx
    normalization_user_artist_rating = pd.DataFrame(user_artist_rating)
    ms = MinMaxScaler()
    ccc = pd.DataFrame(ms.fit_transform(close_centrality_drop))
    ccc.index = close_centrality_drop.index
    ccc.columns = close_centrality_drop.columns

    grs(i, matrix, ccc, normalization_user_artist_rating)

cluster수: 10 	hyper param: 0.84/-0.94뺌 / NDCG: 0.2341499426759837
